In [1]:
# std packages
%matplotlib inline
import numpy as np
import scipy.interpolate as interp
import scipy.signal as sig
import scipy.optimize as opt
import scipy.integrate as integ
import scipy.linalg as sla
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import rc
import h5py as h5
import os, sys
import timeit

plt.rc('figure', figsize=(9, 7))
plt.rcParams.update({'text.usetex': False,
                     'font.family': 'serif',
                     'font.serif': ['Georgia'],
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 2.5,
                     'font.size': 20,
                     'xtick.labelsize': 'large',
                     'ytick.labelsize': 'large',
                     'xtick.direction': 'in',
                     'ytick.direction': 'in',
                     'axes.labelsize': 'large',
                     'axes.titlesize': 'large',
                     'axes.grid': True,
                     'grid.alpha': 0.5,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 17,
                     'legend.framealpha': 0.7,
                     'legend.handletextpad': 0.5,
                     'legend.labelspacing': 0.2,
                     'legend.loc': 'best',
                     'savefig.bbox': 'tight',
                     'savefig.pad_inches': 0.05,
                     'savefig.dpi': 80,
                     'pdf.compression': 9})

sys.path.append('../')

from myConstants import *
import LKlib as LK


In [ ]:
# fixed pars

In [2]:
Mt_Ms = 50
qq = 0.8
chi1, chi2 = 0.7, 0.7

M1 = Mt_Ms / (1.+qq) * Ms
M2 = M1 * qq
Mt = M1 + M2
mu = M1 * M2 / Mt
eta = mu / Mt

S1 = chi1 * G*M1**2./c
S2 = chi2 * G*M2**2./c

r_Mt = G*Mt/c**2.
t_Mt = r_Mt/c
t_Mt_pi = t_Mt * np.pi

S_Mt = G*Mt**2./c

par = np.array([M1, M2, 0, 0, 
                1e3*t_Mt, S_Mt, 0, r_Mt, S_Mt, S_Mt, 
                1, 1])

__, __, __, __, \
t_unit, L_unit, __, \
a_unit, S1_unit, S2_unit, \
__, __ = par

t_gw = LK.get_inst_t_gw_from_a_orb(M1, M2, 500.*r_Mt, 0)
print('%e'%t_gw, '%e'%(t_gw/t_unit))

4.870373e+06 1.977539e+07


In [ ]:
# sample initial cond

In [3]:
a_init = 500.*r_Mt 
L_init = mu * np.sqrt(G*Mt*a_init)

L_v_init = np.array([0, 0, 1]) * L_init

cS1L = np.random.uniform()
cS2L = np.random.uniform()

phiS1 = np.random.uniform() * 2.*np.pi
phiS2 = np.random.uniform() * 2.*np.pi

print(cS1L, phiS1)

sS1L = np.sqrt(1.-cS1L**2.)
sS2L = np.sqrt(1.-cS2L**2.)

S1_v_init = np.array([
    sS1L * np.cos(phiS1), 
    sS1L * np.sin(phiS1), 
    cS1L]) * S1
S2_v_init = np.array([
    sS2L * np.cos(phiS2), 
    sS2L * np.sin(phiS2), 
    cS2L]) * S2

a_nat_init = a_init / a_unit
L_nat_v_init = L_v_init / L_unit
e_v = np.zeros(3)
S1_nat_v_init = S1_v_init / S1_unit
S2_nat_v_init = S2_v_init / S2_unit

y_nat_init = np.hstack([
    a_nat_init, \
    L_nat_v_init, e_v, \
    S1_nat_v_init, S2_nat_v_init])

dy_nat_init = LK.evol_binary(0, y_nat_init, par)

print(y_nat_init)
print(dy_nat_init)

0.29203503952612897 3.7679580348915716
[ 5.00000000e+02  0.00000000e+00  0.00000000e+00  5.52115550e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.67405114e-01
 -1.21128081e-01  6.30939900e-02 -9.05234385e-03 -9.22737100e-02
  1.02579990e-01]
[-2.52839506e-05 -3.24793010e-05  2.48824156e-05 -1.39596623e-07
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.69243766e-05
 -2.34202419e-05 -5.74023917e-08  1.55549244e-05 -1.46217365e-06
  5.74023917e-08]


In [4]:
# functions to find specific separations
def find_a_Mt(t_nat, y_nat, par, 
           a_Mt_trgt=6):
    a_Mt = y_nat[0]
    resi = a_Mt - a_Mt_trgt
    return resi

event1=lambda t_nat_, y_nat_vect_:find_a_Mt(t_nat_, y_nat_vect_, par, a_Mt_trgt=50)
event1.direction = -1
event1.terminal = False

event2=lambda t_nat_, y_nat_vect_:find_a_Mt(t_nat_, y_nat_vect_, par, a_Mt_trgt=25)
event2.direction = -1
event2.terminal = False

event3=lambda t_nat_, y_nat_vect_:find_a_Mt(t_nat_, y_nat_vect_, par, a_Mt_trgt=10)
event3.direction = -1
event3.terminal = False

event4=lambda t_nat_, y_nat_vect_:find_a_Mt(t_nat_, y_nat_vect_, par, a_Mt_trgt=6)
event4.direction = -1
event4.terminal = True


# function to do the integration
int_func = lambda t_nat_, y_nat_vect_: LK.evol_binary(t_nat_, y_nat_vect_, par)

t_run0 = timeit.default_timer()
sol = integ.solve_ivp(int_func, \
        t_span=(0, 1e9), y0=y_nat_init, rtol=1e-12, atol=1e-12, \
        events=[event1, event2, event3, event4])
t_run1 = timeit.default_timer()

print(t_run1 - t_run0)

13.545427265000004


In [5]:
sol.y_events

[array([[ 5.00000000e+01,  1.76255414e-01, -6.32408509e-02,
          1.73587166e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -1.95754441e-01,  5.64464053e-02,
          7.19120137e-02, -4.01458326e-02, -6.41989803e-02,
          1.15697189e-01]]),
 array([[ 2.50000000e+01,  3.17485139e-02, -1.16671116e-01,
          1.22863249e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -1.74104329e-01, -1.10565711e-02,
          1.27447128e-01,  9.92303308e-02,  7.59138702e-02,
          5.92407147e-02]]),
 array([[10.        , -0.06404628, -0.1038199 ,  0.77122154,  0.        ,
          0.        ,  0.        ,  0.14171709, -0.01931526,  0.16192752,
         -0.10383525,  0.08414397,  0.03545516]]),
 array([[ 5.99999999e+00, -2.54001421e-02, -5.79989202e-03,
          6.04250851e-01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -1.08859622e-01,  6.18406876e-02,
          1.76075687e-01,  1.08370939e-01, -8.49034011e-02,
          1.28914157e-0

In [6]:
len(sol.y_events)

4